<a href="https://colab.research.google.com/github/OwiyeD/Datawarehousing/blob/master/scrape_whiskydata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scraping data from whisky ecomerce website

Using requets to get the resources from https://www.thewhiskyexchange.com/c/35/japanese-whisky

Use BeautifulSoup to parse the html data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Get our url and headers



In [ ]:
baseurl = "https://www.thewhiskyexchange.com"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'}

Make the resource request and parse the html data therefore generating a list of all products

In [ ]:
k = requests.get('https://www.thewhiskyexchange.com/c/35/japanese-whisky').text
soup=BeautifulSoup(k,'html.parser')
productlist = soup.find_all("li",{"class":"product-grid__item"})
print(productlist)

[<li class="product-grid__item"><a class="product-card" href="/p/37326/akashi-red" onclick="_gaq.push(['_trackEvent', 'Products-GridView', 'click', '37326 : Akashi Red'])" title="Akashi Red"><div class="product-card__image-container"><img alt="Akashi Red" class="product-card__image lazy" data-original="https://img.thewhiskyexchange.com/480/japan_aka4.jpg" height="4" src="https://img.thewhiskyexchange.com/ph.png" width="3"/></div><div class="product-card__content"><p class="product-card__name"> Akashi Red </p><p class="product-card__meta"> 50cl / 40% </p></div><div class="product-card__data"><p class="product-card__price"> £30.45 </p><p class="product-card__unit-price"> (£60.90 per litre) </p></div></a></li>, <li class="product-grid__item"><a class="product-card" href="/p/37325/suntory-torys-classic" onclick="_gaq.push(['_trackEvent', 'Products-GridView', 'click', '37325 : Suntory Torys Classic'])" title="Suntory Torys Classic"><div class="product-card__image-container"><img alt="Suntor

In [ ]:
productlinks = []
for product in productlist:
        link = product.find("a",{"class":"product-card"}).get('href')                 
        productlinks.append(baseurl + link)


Get a link to every product's page

In [ ]:
productlinks = []
for x in range(1,6):  
 k = requests.get('https://www.thewhiskyexchange.com/c/35/japanese-whisky?pg={}&psize=24&sort=pasc'.format(x)).text  
 soup=BeautifulSoup(k,'html.parser')  
 productlist = soup.find_all("li",{"class":"product-grid__item"})
 
 for product in productlist:
   link = product.find("a",{"class":"product-card"}).get('href')
   productlinks.append(baseurl + link)

In [ ]:
<div class="product-main__image-container">
<img src="https://img.thewhiskyexchange.com/900/japan_sun15.jpg" alt="Suntory Toki" class="product-main__image" width="3" height="4">
</div>

In [ ]:
data=[]
for link in productlinks:
    f = requests.get(link,headers=headers).text
    hun=BeautifulSoup(f,'html.parser')

    try:
        price=hun.find("p",{"class":"product-action__price"}).text.replace('\n',"")
    except:
        price = None

    try:
        about=hun.find("div",{"class":"product-main__description"}).text.replace('\n',"")
    except:
        about=None

    try:
        rating = hun.find("div",{"class":"review-overview"}).text.replace('\n',"")
    except:
        rating=None

    try:
        name=hun.find("h1",{"class":"product-main__name"}).text.replace('\n',"")
    except:
        name=None
    try:
      image_class = h.find("div", {"class": "product-main__image-container"})
      image = image_class.find('img')
      if image not in data:
        image = image.get('src')
    except:
      image=None

    whisky = {"name":name,"price":price,"rating":rating,"about":about, "image":image}

    data.append(whisky)

In [ ]:
result = []
for li in productlinks:
  f = requests.get(li,headers=headers).text
  h=BeautifulSoup(f,'html.parser')
  try:
    image = h.find("div", {"class": "product-main__image-container"})
    name = image.find('img')
    if name not in result:
      result.append(name.get('src'))
  except:
    name=None
result

In [ ]:
df = pd.DataFrame(data)

df.head(10)

,name,price,rating,about,image
0,Akashi Red,£30.45,4(1 Review),Akashi Red is made at the Eigashima (aka White...,https://img.thewhiskyexchange.com/900/japan_ha...
1,Suntory Torys Classic,£30.45,3.5(2 Reviews),Suntory Torys Classic is a light and easy-drin...,https://img.thewhiskyexchange.com/900/japan_ha...
2,Suntory Toki,£31.95,4(32 Reviews),Toki is a blended whisky from Suntory's three ...,https://img.thewhiskyexchange.com/900/japan_ha...
3,Tokinoka White Blended Whisky,£32.95,None,A Japanese blended whisky from White Oak disti...,https://img.thewhiskyexchange.com/900/japan_ha...
4,Akashi Blended Whisky,£32.95,3.5(19 Reviews),A lesser-known whisky outside the local Japane...,https://img.thewhiskyexchange.com/900/japan_ha...
5,Hatozaki Blended Japanese Whisky,£33.95,3(3 Reviews),Named after the oldest stone lighthouse in Jap...,https://img.thewhiskyexchange.com/900/japan_ha...
6,Suntory TokiGlass Pack,£34.95,None,"A lovely gift set from Suntory, including a bo...",https://img.thewhiskyexchange.com/900/japan_ha...
7,Nikka Days,£38.95,5(2 Reviews),"A vibrant addition to the Nikka range, Nikka D...",https://img.thewhiskyexchange.com/900/japan_ha...
8,Mars Kasei Blended Whisky,£42.95,5(1 Review),"Mars Kasei blended whisky is a soft, rounded w...",https://img.thewhiskyexchange.com/900/japan_ha...
9,Nikka From the Barrel,£42.95,4.5(108 Reviews),Nikka from the Barrel is big-boned and burstin...,https://img.thewhiskyexchange.com/900/japan_ha...


In [ ]:
!pip install Selenium

     |████████████████████████████████| 904 kB 7.3 MB/s 


In [ ]:
import hashlib
import io 
from pathlib import Path 
from PIL import Image 
from selenium import webdriver

# https://oxylabs.io/blog/scrape-images-from-website
# 